In [18]:
import numpy as np
import pandas as pd
import pickle

case_pqtl_path = "/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_known_loci/known_pro_assoc/06.pqtl/CTEPH_pqtl_result.pkl"
control_pqtl_path = "/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_known_loci/known_pro_assoc/06.pqtl/NAGA_pqtl_result.pkl"
apt_summary_path = "/LARGE0/gr10478/b37974/Pulmonary_Hypertension/Proteome/Soma_QC_re/Results/05.seqid2uniprot/PHOMday0_vs_NAGA.summary.uniprot.csv"

case_pqtl = pd.read_pickle(case_pqtl_path)
control_pqtl = pd.read_pickle(control_pqtl_path)
apt_summary = pd.read_csv(apt_summary_path, index_col='SeqId')

In [19]:
model_name = 'additive'

In [20]:
def extract_col_dict(dict, model_name):
    extracted_dict = {}
    for snpid, df in dict.items():
        required_columns = {'SeqId', f'{model_name}_beta', f'{model_name}_pvalue', f'{model_name}_fdr', f'{model_name}_sd'}
        # print(f'Checking {snpid}: required columns {required_columns}')
        # print(f'Available columns: {set(df.columns)}')
        if required_columns.issubset(df.columns):
            extracted_df = df[['SeqId', f'{model_name}_beta', f'{model_name}_pvalue', f'{model_name}_fdr', f'{model_name}_sd']].set_index('SeqId')
            extracted_dict[snpid] = extracted_df
        else:
            print(f'{snpid} does not have {model_name} columns')
    return extracted_dict

case_pqtl_additive = extract_col_dict(case_pqtl, model_name)
control_pqtl_additive = extract_col_dict(control_pqtl, model_name)

In [21]:
apt_reserved = apt_summary[apt_summary['status'] == 'Reserved']

def merge_pqtl_apt(dict, apt_df):
    merged_dict = {}
    for snpid, df in dict.items():
        merged_df = pd.merge(df, apt_df, left_index=True, right_index=True, how='inner')
        merged_df = merged_df[[f'{model_name}_beta', f'{model_name}_pvalue', f'{model_name}_fdr', f'{model_name}_sd', 'UniProt', 'EntrezGeneID', 'EntrezGeneSymbol']]
        merged_dict[snpid] = merged_df
    return merged_dict

case_pqtl_additive_apt = merge_pqtl_apt(case_pqtl_additive, apt_reserved)
control_pqtl_additive_apt = merge_pqtl_apt(control_pqtl_additive, apt_reserved)

In [22]:
def extract_and_merge(case_dict, control_dict):
    merged_dict = {}
    for key in case_dict.keys():
        if key in control_dict:
            case_df = case_dict[key][[f'{model_name}_beta', 'UniProt', 'EntrezGeneID', 'EntrezGeneSymbol']].rename(columns={f'{model_name}_beta': f'case_{model_name}_beta'})
            control_df = control_dict[key][[f'{model_name}_beta']].rename(columns={f'{model_name}_beta': f'control_{model_name}_beta'})
            merged_df = pd.merge(case_df, control_df, left_index=True, right_index=True, how='inner')
            merged_df = merged_df[[f'case_{model_name}_beta', f'control_{model_name}_beta', 'UniProt', 'EntrezGeneID', 'EntrezGeneSymbol']]
            merged_dict[key] = merged_df
    return merged_dict

merged_pqtl_additive_apt = extract_and_merge(case_pqtl_additive_apt, control_pqtl_additive_apt)

In [23]:
merged_pqtl_additive_apt

{'chr1:169514323:T:C':            case_additive_beta  control_additive_beta UniProt EntrezGeneID  \
 SeqId                                                                       
 X10000-28           -0.191170              -0.075898  P43320         1415   
 X10001-7             0.219162              -0.024766  P04049         5894   
 X10003-15           -0.022429               0.042779  P51814         7592   
 X10006-25            0.082896               0.081817  P19419         2002   
 X10008-43           -0.122206              -0.182256  P43080         2978   
 ...                       ...                    ...     ...          ...   
 X9993-11            -0.003772               0.046310  O43296         9422   
 X9994-217           -0.006162               0.390207  P51164          496   
 X9995-6              0.601038               0.065562  P33316         1854   
 X9997-12             0.377165               0.178379  Q92575        23190   
 X9999-1              0.127888            

In [24]:
import os

os.makedirs('data', exist_ok=True)

with open('data/merged_pqtl_additive_apt.pkl', 'wb') as f:
    pickle.dump(merged_pqtl_additive_apt, f)